<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Mortgage Calculator chatbot using Generative AI with Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233c'><b>Introduction:</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the Mortgage Calculator chatbot using Generative AI demo, the combination of <b>RAG, Langchain, and LLM models</b> allows users to ask queries in layman's terms, retrieve relevant information from the Vantage tables, and generate accurate and concise answers based on the retrieved data. This integration of retrieval-based and generative-based approaches provides a powerful tool for extracting knowledge from structured sources and delivering user-friendly responses.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this demo we will build Generative Question-Answering using LangChain, a powerful library for working with LLMs like GPT-3.5, GPT-4, Mistral 7B, Mixtral 8x22B, etc. and JumpStart in ClearScape notebooks, a system is built where users can ask business questions in natural English and receive answers with data drawn from the relevant databases.</p>


<center><img src="images/header.png" alt="mortgage calc"  width=800 height=800/></center>

<br>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Before going any farther, let's get a better understanding of RAG, LangChain, and LLM.</p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'><li> <b>Retrieval-Augmented Generation (RAG):</b></li></ul>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> &emsp;  &emsp;RAG is a framework that combines the strengths of retrieval-based and generative-based approaches in question-answering systems.It utilizes both a retrieval model and a generative model to generate high-quality answers to user queries. The retrieval model is responsible for retrieving relevant information from a knowledge source, such as a database or documents. The generative model then takes the retrieved information as input and generates concise and accurate answers in natural language.</p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'><li> <b>Langchain:</b></li></ul>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> &emsp;  &emsp; LangChain is a framework that facilitates the integration and chaining of large language models with other tools and sources to build more sophisticated AI applications. LangChain does not serve its own LLMs; instead, it provides a standard way of communicating with a variety of LLMs, including those from OpenAI and HuggingFace. LangChain accelerates the development of AI applications with building blocks. We learn the leverage the following building blocks in this notebook:</p>
 
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li> <b> LLMs</b> – LangChain's <code>llm</code> class is designed to provide a standard interface for all LLM it supports.   </li>
    <li> <b> PromptTemplate</b>  - LangChain’s <code>PromptTemplate</code> class are predefined structures for generating prompts for LLM’s. They can be reused across different LLM's.</li>
    <li> <b> Chains</b> – When we build complex AI applications, we may need to combine multiple calls to LLM’s and to other components  LangChain’s <code>chain</code> class allows us to link calls to LLM’s and components. The most common type of chaining in any LLM application is combining a prompt template with an LLM and optionally an output parser. </li>
</ol>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'><li> <b> LLM Models (Large Language Models):</b></li></ul>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> &emsp;  &emsp; LLM models refer to the large-scale language models that are trained on vast amounts of text data.
These models, such as <code>GPT-3.5 (Generative Pre-trained Transformer)</code>, <code>GPT-4</code> , <code>LLaMA 2</code> ,<code> Google's Gemini 1.5 pro</code> , <code>Anthropic Claude 3.0</code> , <code>Mistral 7B</code> ,<code>Mixtral 8x22B.</code> etc. are capable of generating human-like text responses. LLM models have been pre-trained on diverse sources of text data, enabling them to learn patterns, grammar, and context from a wide range of topics. They can be fine-tuned for specific tasks, such as question-answering, natural language understanding, and text generation.
LLM models have achieved impressive results in various natural language processing tasks and are widely used in AI applications for generating human-like text responses.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Why Vantage?</b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Chatbots, powered by Machine Learning and Artificial Intelligence, offer numerous advantages in the context of improving customer experience, particularly in the banking and financial institutions sector. Traditional methods of visiting a bank or financial institution in person and providing extensive information, which is then manually reviewed by an officer, are often time-consuming, error-prone, and rely on outdated manual processes.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Vantage provides these same proven capabilities to search user details, proving personalized offers, helping to calcaulate mortgage,etc, integrated as native ClearScape Analytic functions. This allows organizations to drastically reduce human workforce by chatbot, while allowing for much more user friendly and easy interations.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233c'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Configuring the environment</li>
    <li>Connect to Vantage</li>
    <li>Data Exploration</li>
    <li>Setup LLM</li>
    <li>Launch the Chatbot</li>
    <li>Cleanup</li>
</ol>

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233c'>1. Configuring the environment</b>

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial;color:#00233C'><i><b>Note:</b>The installation of the required libraries will take approximately <b>4 to 5 minutes</b> for the first-time installation. However, if the libraries are already installed, the execution will complete within 5 seconds.</i></p>

In [ ]:
%%capture
!pip install -r requirements.txt

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <i>The above statements will install the required libraries to run this demo. To gain access to installed libraries after running this, restart the kernel.</i></p>

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial;color:#00233C'><i><b>Note:</b> The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed. If you uncomment those installs, be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b>0 0</b></i></p></div>

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial;color:#00233C'><i><b>Note:</b> To ensure that the Chatbot interface reflects the latest changes, please reload the page by clicking the 'Reload' button or pressing F5 on your keyboard for <b>first-time only</b> This will update the notebook with the latest modifications, and you'll be able to interact with the Chatbot using the new libraries.</i></p></div>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Import the required libraries</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import os
import pandas as pd

# teradata lib
from teradataml import *

# LLM
from langchain_community.agent_toolkits import create_sql_agent
from langchain.tools import tool
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)

from langchain.memory import (
    ConversationSummaryBufferMemory,
)
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import AgentType

from langchain.agents import AgentExecutor
from langchain_core.runnables import RunnablePassthrough

# Suppress warnings
warnings.filterwarnings("ignore")
display.print_sqlmr_query = False
display.suppress_vantage_runtime_warnings = True

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233c'>2. Connection to Vantage and OpenAI</b>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>2.1 Get the OpenAI API key</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In order to utilize this demo, you will need an OpenAI API key. If you do not have one, please refer to the instructions provided in this guide to obtain your OpenAI API key: </p>

[Openai_setup_api_key_guide](..//Openai_setup_api_key/Openai_setup_api_key.md)

In [ ]:
import getpass

# enter your openai api key
api_key = getpass.getpass(prompt="\n Please Enter Openai api key: ")

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>2.2 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
execute_sql('''SET query_band='DEMO= Mortgage_Calculator_Python.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>2.3 Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the cell below, we are synthetically generating a few records for the <b>Customer</b> and <b>Interest</b> tables. This will allow us to test the functionality of our Chatbot with a small set of data before loading the entire dataset.For the Customer table, we are generating 5 records with randomized names, emails, and phone numbers, account details, etc.  We are also assigning a unique customer ID to each record. For the Interest table, we are generating 5 records with randomized interest rates based on minimum and maximum of credit score.  We are also assigning a unique interest ID to each record.By generating this sample data, we can quickly test the application's ability to display customer information and interests.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In addition to this, we are loading data for <b>RealEstate, Locality, NearbyLocality, School, and ShoppingCenter</b>. We take RealEstate data from Kaggle, clean it first, and then add it to Vantage tables in the below code.</p>

<p style='font-size:16px;font-family:Arial;color:#00233C'>We iterate through a dictionary mapping file names to primary keys, read corresponding CSV files using Pandas, and copy the data to Vantage using the teradataml <b>copy_to_sql</b> function. </p>

In [ ]:
files_dict = {
    "Customer": "CustomerID",
    "Interest": "ID",
    "RealEstate": "PropertyID",
    "Locality": "LocalityID",
    "NearbyLocality": "LocalityID",
    "School": "SchoolID",
    "ShoppingCenter": "StoreID",
}


print("-" * 25, "data ingestion started", "-" * 25)
for file in files_dict:
    df = pd.read_csv(os.path.join("./data/", f"{file}.csv"))
    print("Copying data to Vantage for: ", file)
    print(f"Data information: {df.shape}")
    copy_to_sql(
        df, table_name=file, primary_index=files_dict[file], if_exists="replace"
    )
print("-" * 25, "data ingestion completed", "-" * 25)

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233c'>3. Data Exploration</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The goal of the Mortgage Chatbot is to provide a user-friendly interface for individuals to quickly and easily find personalized mortgage information. The chatbot aims to streamline the mortgage process by allowing users to ask questions in natural language and receive accurate, concise answers and calculations. By integrating with Vantage, the chatbot can retrieve relevant data and perform calculations to provide users with tailored mortgage options, empowering them to make informed decisions and save time in their home buying journey. </p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In our database, we have total seven tables, we can categorize it into two: </p>
    <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li> Financial tables</li>
                <li> Property tables</li>
    </ol>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'><li> <b>Financial tables:</b></li></ol>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> There are two tables: <b>customer</b> and <b>interest</b>. The customer table contains personal and financial information, while the interest table has details on credit score ranges and their corresponding interest rates.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The data is synthetically generated, with a few rows of customer data, including name, address, email, account number, balance, income, and other relevant information.<p/>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Each row represents a snapshot of data captured at a specific point in time, and each column represents a different variable. The input dataset is as follows:</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> 
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Customer data</b> i.e. first-name, last-name, date of birth, address, city, account number, Balance, branch id, account status, income etc.</li>
    <li><b>Interest data</b>: min credit score, max credit score, interest rate.</li>
</ol>
</p>


<ol style = 'font-size:16px;font-family:Arial;color:#00233C', start=2><li> <b>Property tables:</b></li></ol>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have five tables that cover various aspects, ranging from real estate to locality details. For instance, let's take the locality of <b>Larrakeyah</b>, situated <b>9.0 km away from the airport</b>, boasting a <b>livability score of 4.3.</b> In Larrakeyah, we find <b>three schools:</b> St John's Catholic College, Larrakeyah Primary School, and Essington International School. Moreover, there are <b>two shopping centers:</b> Larrakeyah Shopping Complex and Larrakeyah Supermart.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see each row as a snapshot of data captured at a specific point in time, and each column represents a different variable. This is our input dataset.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> 
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
   <li><b>RealEstate data</b>:LocalityID, PropertyID, PropertyType, BuildingSize, LandSize, PreferredSize, OpenDate, ListingAgency, Price, Address, City, State, ZipCode, Phone, 
       BedroomCount, BathroomCount, ParkingCount</li>
    <li><b>Locality data</b>: LocalityID, LocalityName, LocalityDescription, DistanceFromAirport, LivabilityScore</li>
    <li><b>NearbyLocality data</b>: LocalityID, LocalityName, NearbyLocalityID</li>
    <li><b>School data</b>: SchoolID, SchoolName, LocalityID</li>
    <li><b>ShoppingCenter </b>: StoreID, StoreName, Type, LocalityID</li>
</ol>
</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The source data from <a href="https://www.kaggle.com/datasets/thedevastator/australian-housing-data-1000-properties-sampled">kaggle</a> is loaded in Vantage with table named <i>RealEstate</i>.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b><i>*Please scroll down to the end of the notebook for detailed column descriptions of the dataset.</i></b></p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>3.1 Examine the Customer and Interest table</b></p>    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's look at the sample data in Customer table.</p>

In [ ]:
tdf_customer = DataFrame("Customer")

print("Data information: \n", tdf_customer.shape)
tdf_customer.sort("CustomerID")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>There are 5 records in all, and there are 18 variables.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now, let's delve into the interest table. Here, we determine interest rates based on customers' minimum and maximum credit scores.</p>

In [ ]:
tdf_interest = DataFrame("Interest")

print("Data information: \n", tdf_interest.shape)
tdf_interest

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's examine that interest rate in light of a person's credit score. Here, we may determine the interest rate for each individual consumer by examining the credit score range min and max from the interest table.</p>

In [ ]:
q = """
    SELECT c.FirstName, c.LastName, c.CreditScore, i.InterestRate 
    FROM Customer c 
    JOIN Interest i ON c.CreditScore BETWEEN i.MinCreditScore AND i.MaxCreditScore
    WHERE c.CustomerID = 'CID3058245';
"""

DataFrame.from_query(q)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We can observe that Customer John Doe possesses a credit score of 720, falling within the range of 670 to 739. Therefore, we will offer him a 3.9% interest rate.</p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>3.2 Examine the property tables</b></p>    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's look at the sample data in <b>RealEstate</b> table first.</p>

In [ ]:
tdf_real_estate = DataFrame("RealEstate")

print("Data information: \n", tdf_real_estate.shape)
tdf_real_estate.head(5)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We can see that we have ample details about the property here, such as property type, land size, bedroom count, contact details, price, full address, etc. These are crucial details for customers when we search for properties.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next, let's examine the sample data in our <b>Locality</b> table.</p>

In [ ]:
tdf_locality = DataFrame("Locality")

print("Data information: \n", tdf_locality.shape)
tdf_locality.head(5)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the above table, we can gain a better understanding of the locality, including its distance from the airport and livability score.</p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Thirdly, let's review the sample data in our <b>NearbyLocality</b> table.</p>

In [ ]:
tdf_nearby_locality = DataFrame("NearbyLocality")

print("Data information: \n", tdf_nearby_locality.shape)
tdf_nearby_locality.head(5)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next, let's review the sample data in our <b>School</b> table.</p>

In [ ]:
tdf_school = DataFrame("School")

print("Data information: \n", tdf_school.shape)
tdf_school.head(5)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next, let's review the sample data in our <b>ShoppingCenter</b> table.</p>

In [ ]:
tdf_shopping_center = DataFrame("ShoppingCenter")

print("Data information: \n", tdf_shopping_center.shape)
tdf_shopping_center.head(5)

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>3.3 Get database schema</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Build a consolidated view of Table Data Catalog by combining metadata stored for the database and table.</p>

In [ ]:
from langchain.sql_database import SQLDatabase

#  Create the vantage connection
db = SQLDatabase(eng)
database = "demo_user"

main_d = {
    "Customer": tdf_customer.columns,
    "Interest": tdf_interest.columns,
    "RealEstate": tdf_real_estate.columns,
    "Locality": tdf_locality.columns,
    "NearbyLocality": tdf_nearby_locality.columns,
    "School": tdf_school.columns,
    "ShoppingCenter": tdf_shopping_center.columns,
}


def get_db_schema():
    table_dicts = []
    for k in main_d:
        table_dicts.append(
            {
                "database_name": database,
                "table_name": k,
                "column_names": main_d[k],
            }
        )

    database_schema_string = "\n".join(
        [
            f"Database: {table['database_name']}\nTable: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
            for table in table_dicts
        ]
    )

    return database_schema_string

In [ ]:
database_schema = get_db_schema()
print(database_schema)

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233c'>4. Setup LLM </b>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>4.1 Connect to database using SQLAlchemy and Initialize the Large Language Model</b></p>    

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Under the hood, LangChain uses SQLAlchemy to connect to SQL database. The SQLDatabaseToolkit can therefore be used with any SQL dialect supported by SQLAlchemy, such as Teradata Vantage, MS SQL, MySQL, MariaDB, PostgreSQL, Oracle SQL, and SQLite. Please refer to the <a href="https://docs.sqlalchemy.org/en/20/"> SQLAlchemy documentation</a> for more information about requirements for connecting to your database. The SQLDatabaseToolkit builds off of SQLDatabaseChain and is designed to answer more general questions about a database, as well as recover from errors.</p>


<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial;color:#00233C'><i><b>Important Note:</b> The code below establishes a database connection for data sources and Large Language Models. Please note that the solution will only work if the database connection for your sources is defined in the cell below. In addition to this, to use OpenAI models, we have to set OpenAI API key to environment variable.</i></p></div>

In [ ]:
from langchain_openai import ChatOpenAI

# OpenAI API
os.environ["OPENAI_API_KEY"] = api_key

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this demo we will be using OpenAI’s <code>gpt-3.5-turbo-0125</code> model as LLM. To view list of available models on OpenAI <a href='https://platform.openai.com/docs/models'>click here</a></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In OpenAI's language models, the <b>temperature</b> parameter controls the randomness of the generated text. It affects the diversity and creativity of the model's responses. It is always a number between 0 and 1. A temperature of 0 means the responses will be very straightforward, almost deterministic (meaning you almost always get the same response to a given prompt). A temperature of 1 means the responses can vary wildly.</p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>A higher temperature value, such as 1.0, increases the randomness and diversity of the generated output. This can lead to more varied and surprising responses, but it may also result in less coherence and occasional nonsensical outputs. A higher temperature means that the model might select a word with slightly lower probability, leading to more variation, randomness and creativity. A very high temperature therefore increases the risk of <b>hallucination</b>, meaning that the model starts selecting words that will make no sense or be off the topic.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>On the other hand, a lower temperature value, such as 0.2 or below, reduces randomness and makes the model's output more focused and deterministic. The generated text is likely to be more conservative, sticking closely to patterns observed in the training data. A temperature of 0 means roughly that the model will always select the highest probability word.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Choosing an appropriate temperature value depends on the desired output. Higher temperatures can be useful for creative tasks or brainstorming, while lower temperatures are preferred when you need more control over the output, such as when generating specific responses or following a particular style.</p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>4.2 Setup SQLAgent</b></p> 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Before jumping into the chatbot, let us first understand what is an agent and why it might be preferred over a simple SQLChain.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>An agent is a component that has access to a suite of tools, including a Large Language Model (LLM). Its distinguishing characteristic lies in its ability to make informed decisions based on user input, utilizing the appropriate tools until it achieves a satisfactory answer. For example in the context of text-to-SQL, the LangChain SQLAgent will not give up if there is an error in executing the generated SQL. Instead, it will attempt to recover by interpreting the error in a subsequent LLM call and rectify the issue. Therefore, in theory, SQLAgent should outperform SQLChain in productivity and accuracy.</p>
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We can think of agents as enabling tools for LLMs. Like how a human would use a calculator for maths or perform a Google search for information — agents allow an LLM to do the same thing.</p>

In [ ]:
# define the tool
@tool
def generate_sql(query: str) -> str:
    """Given an input question, first create a syntactically correct Teradata-style query to run, then look at the results of the query and return the answer."""
    agent_executor = create_sql_agent(
        llm=llm,
        toolkit=SQLDatabaseToolkit(db=db, llm=llm),
        verbose=True,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        max_iterations=8,
    )
    return agent_executor.run(query)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the code above, we're initializing SQLAgent by passing in LLM, SQLDatabaseToolkit, database, and agent type as ZeroShotReactAgent. Agents use an LLM to determine the best course of action and execute it. An action can either be using a tool and observing its output, or responding to the user. <a href='https://python.langchain.com/docs/modules/agents/agent_types/'>Here</a> are the agents available in LangChain.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The agent used here is a <b>zero-shot-react-description</b> agent. Zero-shot means the agent functions on the <b>current action only</b> — it has no memory. It uses the <a href='https://arxiv.org/pdf/2210.03629.pdf'>ReAct</a> framework to decide which tool to use, based solely on the tool’s description.</p>

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233c'>5. Launch the Chatbot</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this demo we are using ChatOpenAI model with Memory. This advanced technology allows us to store and recall conversations, enabling our chatbot to provide more personalized and informed responses.As a mortgage advisor, our chatbot is trained to assist with a wide range of loan-related inquiries. Whether you're looking to purchase a new home, refinance an existing loan, or simply have questions about the mortgage process, our chatbot is here to help.To begin, we'll set the prompt for our chatbot to work as a mortgage advisor. This will enable it to provide tailored advice and guidance based on your unique needs and circumstances.</p>

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial;color:#00233C'><i><b>Note:</b>Please note that our chatbot is specifically designed to address questions related to mortgage loans. If you have a question outside of this scope, we kindly ask that you refrain from asking it. This will help us provide you with the most accurate and efficient assistance possible.</i></p>
</div>


In [ ]:
# Current user
CustomerID = 1001

tools = [generate_sql]
functions = [format_tool_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0, streaming=True).bind(functions=functions)
memory = ConversationSummaryBufferMemory(
    return_messages=True, memory_key="chat_history", llm=llm
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You'll be working as property and mortgage advisor and I will play the role of the user.""",
        ),
        (
            "system",
            """First act as property advisor, once you suggest the property, ask user Do you like this or shall I suggest more? Once the user say he likes it, 
            then only perform a role of mortgage advisor.""",
        ),
        (
            "system",
            """As a property advisor, you have direct access of property data includes tables like RealEstate, Locality, NearbyLocality,  ShoppingCenter, School.""",
        ),
        (
            "system",
            """As a property advisor first Ask a series of questions related to property like preferred locality, number of bedrooms, etc. Once user select the property then only ask for mortgage related questions like monthly income, credit score, bank balance etc, 
            Do not ask same question more than one time.
            Always give property options with details like Property ID, Property Type, Building Size, Price, Address,Bedroom Count""",
        ),
        (
            "system",
            """Second, act as a mortgage advisor for the bank you have direct access to the banks data for user. The banks data includes tables like Customer and Interest""",
        ),
        (
            "system",
            """Once you suggest the property to the user, then ask about mortgage of property. You can ask question like below one by one:
            1st question: Income: What is your total annual income, including any additional sources of income?
            2nd question: What is your employment status?
            3rd: What's your credit score? I can help you find out which interest rates you qualify for.""",
        ),
        (
            "system",
            """Ask questions sequentially - pausing between each question to wait for a response before proceeding to the next.""",
        ),
        (
            "system",
            """Start the chat with greeting and ask the questions to user as mentioned below. Greet user by starting the message: 
              ## Welcome to Property and Mortgage Chatbot! \n
              I'm here to provide you with expert advice and answers. \n
              ### Let's get started! \n
              Could you please share your property preferences with me?""",
        ),
        (
            "system",
            """Do not ask all the questions together, ask one by one.""",
        ),
        (
            "system",
            """"Provide a succinct summary paragraph of these details as a response to the client but don't show the method of calculation. 
            Follow with a formatted table of monthly amounts (gross income, tax obligation, mortgage obligation, expenses, remaining disposable income).
            Also provide best property options based on user's data.""",
        ),
        (
            "system",
            """Provide an opinion as to whether the mortgage is affordable and what the maximum borrowing amount could be""",
        ),
        (
            "system",
            """Do not append 'Query:' to SQLQuery. Do not remove dashes from the Query. Display SQLResult after the query is run in plain english that users can understand. 
            Provide answer in simple english statement.
            Only use the following Column names: \n
             """
            + database_schema
            + """ """,
        ),
        (
            "system",
            """Given an input question, create a syntactically correct  query to run, then look at the results of the query and return the answer.
            Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 3 results by using SELECT TOP 3, note that LIMIT function does not works in Teradata DB. """,
        ),
        (
            "system",
            f"""If the user fails to provide a response or says 'I don't know' for a question, automatically call the 'generate_sql' function to retrieve the answer from the database. 
            The function should return the relevant answer for the question asked.
            current user: {CustomerID}""",
        ),
        (
            "system",
            """Whenever possible, give the answer in bulleted points and title in markup. like ##""",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
    )
    | prompt
    | model
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=chain,
    tools=tools,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True,
    max_iterations=8,
)

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial;color:#00233C'><i><b>Note:</b>Chatbot is accessing multiple components, including databases and LLMs. This may cause a brief delay in responses. Your patience is appreciated.</i></p>
</div>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This chatbot is designed to help you with any questions you have about loans and mortgages. You can start by typing any words or greeting, like "Hi there!" and the chatbot will respond with a friendly welcome message. </p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here are some ways we can interact with our Property and Mortgage Chatbot: This chatbot is tailored to help you find the ideal property based on your requirements and assists in calculating mortgages accurately.

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Property search: </b></li>
</ol>
    
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Ask a question: </b> Type your question into the chat window, and the chatbot will do its best to provide a helpful response. For example, you could ask:
        <ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
            <li><b>Which is the best locality near the city center?</b></li>
            <li><b>Can you suggest apartments with a land size greater than 300 square meters and featuring at least two bedrooms?</b> </li>
        </ul>
    </li>
    
<li><b>Provide information: </b> The chatbot may ask you for more information to better understand your question or provide a more accurate response. For example, it might ask:
    <ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
            <li><b>Can you suggest good properties within my budget range of $250000 to \$410000?</b> </li>
            <li> <b>Could you please provide me with houses listed by the agency "Ray White - NIGHTCLIFF"?</b> </li>
    </ul>
    </li>

    
<li><b>Get advice: </b>The chatbot can provide guidance on the property search process and offer tips for finding the right properties for your needs. You can ask it questions like:
    <ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
        <li><b>What are the pros and cons of a buying a property on Larrakeyah area?</b> </li>
        <li><b>What are the neighborhood areas within Larrakeyah?</b></li>
        </ul>
    </li>
</ul>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C' start=2>
    <li><b>Mortgage calculation: </b></li>
</ol>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Ask a question: </b> Type your question into the chat window, and the chatbot will do its best to provide a helpful response. For example, you could ask:
        <ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
            <li><b>What is the best Interest rate would you offer to me?</b></li>
            <li><b>What is the current balance in my account?</b> </li>
        </ul>
    
<li><b>Provide information: </b> The chatbot may ask you for more information to better understand your question or provide a more accurate response. For example, it might ask:
    <ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
            <li><b>Do you like any of these properties, or should I suggest more options?</b></li> 
            <li><b>Are you looking to purchase a home or refinance an existing mortgage?</b> </li>
    </ul>

<li><b>Get advice: </b>The chatbot can provide guidance on the mortgage process and offer tips for finding the right loan for your needs. You can ask it questions like:
    <ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
        <li><b>What are the pros and cons of a fixed-rate mortgage?</b></li>
        <li><b>How do I choose the best lender?</b></li>
    </ul>
</ul>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Remember, the Mortgage Chatbot is here to help you, so feel free to ask any questions you have. It's available 24/7, so you can get the information you need at a time that's convenient for you.
Start your conversation with the Mortgage Chatbot now by typing your first message in the chat window! </p>

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial;color:#00233C'><i><b>Note:</b>An example of the tentative output the chatbot will provide at the end is shown here.</i></p>
</div>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Thank you for providing your current monthly mortgage payment. </p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Based on the information you have provided, here is a summary of your financial details:</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
<li>Total annual income: \$850,000</li>
<li>Employment status: Full-time</li>
<li>Credit score: 720</li>
<li>Monthly expenses: \$12,000</li>
<li>Term length: 15 years</li>
<li>Additional assets value: \$100,000</li>
<li>Current monthly mortgage payment: \$5,000</li>
</ol>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now, let me calculate the maximum house price and monthly repayment for principal and interest based on a 5% APR. Please give me a moment.</p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>According to my calculations, the maximum house price you can afford is \$1,500,000. The estimated monthly repayment for principal and interest would be approximately $9,500.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here is a breakdown of your monthly amounts:</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
<li>Gross income: \$70,833</li>
<li>Tax obligation (25%): \$17,708</li>
<li>Mortgage obligation: \$9,500</li>
<li>Monthly expenses: \$12,000</li>
<li>Remaining disposable income: \$31,625</li>
</ol>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Based on these calculations, it appears that the mortgage is affordable for you. However, please keep in mind that this is just an estimate and other factors such as your debt-to-income ratio and credit history may also be considered by the bank.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>If you have any further questions or if there's anything else I can assist you with, please let me know.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's chat with the bot!</p>

In [ ]:
import panel as pn

pn.extension(design="material")

# clear the memory
memory.clear()

def callback(contents, user, instance):
    return agent_executor.invoke({"input": contents})["output"]


pn.chat.ChatInterface(
    callback=callback,
    show_rerun=False,
    show_undo=False,
    show_clear=False,
    width=800,
    height=400,
).servable()

<i>If the chatbot didn't work when you pressed ENTER, on your first time using this demo on your environment, did you use F5 to reload the site? See instructions at the top of the notebook.<br>
If you asked a question and got no reponse after a few minutes, it is possible that you will need to type 0 0 to restart the kernel and re-run the demo. Questions outside the model seem to confuse the chatbot.  </i>

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233c'>5 Cleanup</b>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>5.1 Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Cleanup work tables to prevent errors next time.</p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'> <b>5.2 Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
for t in [
    "Customer",
    "Interest",
    "RealEstate",
    "Locality",
    "NearbyLocality",
    "School",
    "ShoppingCenter",
]:

    try:

        db_drop_table(t)

    except:
        pass

In [ ]:
remove_context()

<b style = 'font-size:28px;font-family:Arial;color:#00233c'>Dataset:</b>

**Customer Table**
- `CustomerID`: unique row customer id
- `FirstName`: customer first name
- `LastName`: customer last name
- `DateOfBirth`: customer birth date
- `Gender` : customer gender (categorical: "M","F",)
- `Address` : customer full address
- `City`: city of customer (categorical: 'Almond','Geneva')
- `State`: state of customer (categorical: 'NY','TX')
- `Country`: country of customer (categorical: 'USA')
- `Email`: customer email
- `PhoneNumber`: customer phone number
- `AccountNumber`: customer account number
- `AccountType`: customer account type
- `AccountStatus`: customer account status (categorical: 'Active','Inactive')
- `Balance`: account balance, in dollar (numeric)
- `BranchID`: branch id
- `CreditScore`: customer credit score
- `Income`: customer's monthly income, in dollar (numeric)

**Interest Table**
- `ID`: unique row id
- `MinCreditScore`: min credit score
- `MaxCreditScore`: max credit score
- `InterestRate`: rate of interest

**RealEstate**
- `LocalityID`: unique row locality id (Numeric)
- `PropertyID`: unique row property id (Numeric)
- `PropertyType`: type of property being listed (categorical: 'Apartment','House', 'Unit')
- `BuildingSize`: size of the property's building, in square meters. (Numeric)
- `LandSize`: size of the property's land, in square meters. (Numeric)
- `PreferredSize`: preferred size of the property, in square meters. (Numeric)
- `OpenDate`: date that the property was first listed for sale. (Date)
- `ListingAgency`: agency that is listing the property
- `Price`: listing price of the property
- `Address`: property's address
- `City`: city that the property is located in
- `State`: state that the property is located in
- `ZipCode`: zip code that the property is located in
- `Phone`: listing agent's phone number
- `BedroomCount`: number of bedrooms in the property
- `BathroomCount`: number of bathrooms in the property
- `ParkingCount`: number of parking spaces in the property

 **Locality data**
 - `LocalityID`: unique row locality id (Numeric)
 - `LocalityName`: locality name
 - `LocalityDescription`: locality description
 - `DistanceFromAirport`: distance from airport to locality (Numeric)
 - `LivabilityScore`: locality livability score (Numeric)
 
**NearbyLocality data**
- `LocalityID`: unique row locality id (Numeric)
- `LocalityName`: locality name (Numeric)
- `NearbyLocalityID`: Unique row locality id (Numeric)

**School data**
- `SchoolID`: unique row school id (Numeric)
- `SchoolName`: school name
- `LocalityID`: unique row locality id (Numeric)

**ShoppingCenter**
- `StoreID`: unique row store id (Numeric)
- `StoreName`: store name
- `Type`: type of shopping center (categorical: 'Grocery Store','Shopping Mall')
- `LocalityID`: unique row school id (Numeric)


<p style = 'font-size:16px;font-family:Arial;color:#00233c'><b>Dataset source:</b> <a href = 'https://www.kaggle.com/datasets/thedevastator/australian-housing-data-1000-properties-sampled'>kaggle</a></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233c'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>Langchain Python reference: <a href='https://python.langchain.com/docs/get_started/introduction.html'>here</a></li>
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023, 2024. All Rights Reserved
        </div>
    </div>
</footer>